In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
## Read Bitcoin Heist Full Data csv
## And create a set of ransomware addresses 
## To be used in sampling outputs
df_btch = pd.read_csv(r'../input/bitcoinheist-main-data/BitcoinHeistData.csv')
set_ransomware = set(df_btch.loc[df_btch['label'] != "white"]['address'])

In [4]:
## Sampling Outputs
## Criteria: 
## (i) If any of the addresses in a row is ransomware - Include the entire row
## (ii) Otherwise include every 797th row
## (Just a random prime number to scale down the data into managable size)
## Note: We also track the corresponding Transaction Hash in list "list_transactions"
## In order to filter corresponding inputs from input file

path_input = '/kaggle/input'
path_output = '/kaggle/working/'
## List To track the transaction hashes of included rows
list_transactions = []
for root, dir, files in os.walk(path_input):
    for f in files:
        if "output" in f:
            count_line = 0
            with open(os.path.join(root,f)) as fi:
                with open(os.path.join(path_output,f), "w") as fo:
                    for line in fi:
                        cells = line.rstrip().split("\t")
                        list_addresses = [x for x in cells[2:][::2]]
                        in_ransomw = set_ransomware.intersection(list_addresses)
                        if(len(in_ransomw)>0):
                            fo.write(line)
                            transaction_hash = cells[1]
                            list_transactions.append(transaction_hash)
#                             fo.write('\n')
                            continue
                        if (count_line % 797) == 0:
                            fo.write(line)
                            transaction_hash = cells[1]
                            list_transactions.append(transaction_hash)
#                             fo.write('\n')
                        count_line += 1
                    print("done -->  " + f)

In [5]:
## Sampling Inputs
## Criteria: Filter all the rows where corresponding UTXO's parent transaction is icluded in 
## sampled outputs above. Based on the "list_transactions", which we typeCast to set

transactions_set = set(list_transactions)
for root, dir, files in os.walk(path_input):
    for f in files:
        if "input" in f:
            with open(os.path.join(root,f)) as fi:
                with open(os.path.join(path_output,f), "w") as fo:
                    for line in fi:
                        cells = line.rstrip().split("\t")
                        list_transactions = [x for x in cells[2::2]]
                        included_txns = transactions_set.intersection(list_transactions)
                        if len(included_txns)>0:    
                            fo.write(line)
            print("done -->  " + f)
